<a href="https://colab.research.google.com/github/polydeuces32/s-p500-predicter-on-Python-/blob/main/Simons_inspired_backtest_lab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

using a ration of 1.2 ( Quant Backtest Lab )

In [ ]:
import yfinance as yf
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

# === CONFIG ===
investment_amount = 5000
trading_frequency = 'W'  # 'W' for weekly, 'D' for daily
target_sharpe = 1.2

# === 1. Download SPY data ===
spy = yf.download('SPY', start='2015-01-01', end='2024-12-31')
price_column = 'Adj Close' if 'Adj Close' in spy.columns else 'Close'
spy['Price'] = spy[price_column]

# === 2. Resample to weekly ===
spy = spy.resample(trading_frequency).last()
spy['Returns'] = spy['Price'].pct_change()

# === 3. Indicators for mean reversion ===
window = 20
std_factor = 1.5
spy['MA'] = spy['Price'].rolling(window).mean()
spy['STD'] = spy['Price'].rolling(window).std()
spy['LowerBand'] = spy['MA'] - std_factor * spy['STD']

# === 4. Entry/Exit logic ===
spy['Position'] = 0
spy.loc[spy['Price'] < spy['LowerBand'], 'Position'] = 1
spy.loc[spy['Price'] > spy['MA'], 'Position'] = 0
spy['Position'] = spy['Position'].shift(1).fillna(0)

# === 5. Strategy performance ===
spy['Strategy_Returns'] = spy['Position'] * spy['Returns']
spy['Cumulative_Strategy'] = (1 + spy['Strategy_Returns']).cumprod()
spy['Cumulative_Market'] = (1 + spy['Returns']).cumprod()
spy['Strategy_Value'] = spy['Cumulative_Strategy'] * investment_amount
spy['Market_Value'] = spy['Cumulative_Market'] * investment_amount

# === 6. Sharpe Ratio Calculation ===
mean_return = spy['Strategy_Returns'].mean()
std_return = spy['Strategy_Returns'].std()
sharpe_ratio = (mean_return / std_return) * np.sqrt(52)  # annualized weekly Sharpe

# === 7. Plot ===
plt.figure(figsize=(12, 6))
plt.plot(spy.index, spy['Strategy_Value'], label='Mean Reversion Strategy')
plt.plot(spy.index, spy['Market_Value'], label='Buy & Hold (SPY)', linestyle='--')
plt.title('📈 SPY Mean Reversion Strategy vs Market ($5,000 Investment)')
plt.xlabel('Date')
plt.ylabel('Portfolio Value ($)')
plt.legend()
plt.grid(True)
plt.tight_layout()
plt.show()

# === 8. Summary ===
final_strategy = spy['Strategy_Value'].iloc[-1]
final_market = spy['Market_Value'].iloc[-1]
pnl_strategy = final_strategy - investment_amount
pnl_market = final_market - investment_amount

print("\n🧾 Strategy Summary:")
print(f"  ➤ Final Strategy Value: ${final_strategy:,.2f}")
print(f"  ➤ Final Market Value: ${final_market:,.2f}")
print(f"  ➤ Strategy P&L: ${pnl_strategy:,.2f}")
print(f"  ➤ Market P&L: ${pnl_market:,.2f}")
print(f"  ➤ Sharpe Ratio: {sharpe_ratio:.2f} (Target: {target_sharpe})")

if sharpe_ratio >= target_sharpe:
    print("✅ Sharpe ratio goal achieved!")
else:
    print("⚠️ Sharpe ratio below target. Consider tweaking the strategy.")
